In [ ]:

# #Instaliuojame reikalingas bibliotekas
# %pip install datasets
# %pip install -U transformers
# %pip install peft
# !pip install torch
# !pip install trl


# !pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # For quantization(kvantizavimui)
# !pip install bitsandbytes

# # Modulio vertinimui
# !pip install rouge-score



In [ ]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import Dataset
import torch
from huggingface_hub import login
import pandas as pd
from huggingface_hub import login
from trl import SFTConfig, SFTTrainer
from pathlib import Path
from peft import get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score
import numpy as np

login()


In [ ]:

def formatting_prompts_func(row):
      input_text = row["Input"]
      output_text = row["Output"]
      text = sablonas.format(input_text, output_text) + EOS_TOKEN
      return text

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    # Jei yra keli išvesties rezultatai, apskaičiuoti tikslumą kiekvienam išvesties rezultatui
    accuracies = [accuracy_score(labels[:, i], predictions[:, i]) for i in range(labels.shape[1])]
    return {"accuracy": np.mean(accuracies)}  # arba grąžinti kiekvieno išvesties rezultato tikslumą atskirai

for i in range(6):
  # 2: Inicijuojame Tokenizer
  checkpoint = 'google/gemma-2-2b-it'
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)


  # 3: Inicijuojame modelį, 4bit kvantizavimas, nes kartais gradientų nustatymas nesusiderina
  model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    # torch_dtype=torch.bfloat16,
    load_in_4bit=True,  # Įjungiame 4-bit kvantizavimą
    device_map="auto",  # Automatiškai pasirenkama, ant kurio įrenginio veikti, tai yra kuris yra prieinamas (GPU/CPU)

)
  # 4: Pritaikyti LoRA
  lora_configs=[lora_config_1, lora_config_2, lora_config_3, lora_config_4, lora_config_5, lora_config_6]
  lora_config = lora_configs[i]

  # 5: Priderinti LoRA prie modulio
  model = get_peft_model(model, lora_config)

  # 6 Pasirašome duomenų apdorojimo funkciją, kuri suformatuoja duomenis į eilutes
  sablonas = """ Žemiau yra pateikiama instrukcija ir atsakymas:

  ### Klausimas:
  {}

  ### Atsakymas:
  {}"""

  EOS_TOKEN = tokenizer.eos_token # Reikia pridėti, kad nesikartotų tas pats atsakymas
  dataset_df = pd.read_csv("14_apmokymo_klausimai.csv")
  dataset_df['text'] = dataset_df.apply(formatting_prompts_func, axis=1) # Formatuojama eilutėmis ir prideda naują stulpelį "text", kuris vėliau bus pasiekiamas Trainer
  dataset = Dataset.from_pandas(dataset_df)

  # Padalijame duomenis į treniravimo ir įvertinimo rinkinius (pvz., 80% treniravimui, 20% įvertinimui)
  train_size = int(0.8 * len(dataset_df))
  train_df = dataset_df[:train_size]
  eval_df = dataset_df[train_size:]

  # Konvertuojame DataFrame į Hugging Face Dataset
  train_dataset = Dataset.from_pandas(train_df)
  eval_dataset = Dataset.from_pandas(eval_df)


  # 7 Pasižiūrime, kokie sluoksniai bus treniruojami
  # for name, param in model.named_parameters():
  #     print(f"{name} :-------: {param.requires_grad}")

  # 8 Apibrėžiame treniravimo argumentus.
  training_args = SFTConfig(
      output_dir='res',
      packing=False,
      report_to='none',
      optim="adamw_8bit",
      logging_steps=5,
      evaluation_strategy="steps",
      eval_steps=5,
      max_steps=10,
      max_seq_length=128,
      per_device_train_batch_size=4,
      per_device_eval_batch_size=1,
      learning_rate = 2e-4,
    )

  # 9 Inicijuojame Trainer
  trainer = SFTTrainer(
  model=model,
  args=training_args,
  tokenizer=tokenizer,
  train_dataset=train_dataset,
  eval_dataset=eval_dataset,
  dataset_text_field='text',  # Užtikriname, kad teksto laukas bus naudojamas kaip įvestis
  compute_metrics=compute_metrics,  # Pridedame metrikų funkciją
  peft_config=lora_configs[i],  # Naudojame atitinkamą LoRA konfiguraciją
  )

  # 10: Pradedamas "Fine-Tuning" procesas.
  trainer.train()

  # 11: Išsaugomas apmokytas modelis.

  model.save_pretrained(f"./modelis_{[i]}")
  tokenizer.save_pretrained(f"./modelis_{[i]}")
  duomenys = """
    LoRA nustatymai: {}
    Treniravimo argumentai: {}
  """
  formatas = duomenys.format(lora_configs[i], training_args)

  # Modelio pavadinimas ir katalogo nustatymas
  directory = Path(f"./modelis_{[i]}")
  directory.mkdir(parents=True, exist_ok=True)  # Sukuria katalogą, jei jis dar neegzistuoja

  # Nustatome failo kelią ir įrašome konfiguracijos tekstą
  file_path = directory / "konfiguracija.txt"
  with file_path.open("a", encoding="utf-8") as f:
      f.write("\n" + formatas)

  print("Treniruotė baigta ir išsaugota")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss,Accuracy
5,2.270100,2.374937,0.004115
10,0.628400,2.569221,0.016461


Treniruotė baigta! it išsaugota


## Paprastasis čiatas.

## Tikrinama, kaip apsimokė modelis
---
Ši dalis, naudojama testavimui su apmokymo_klausimai.csv



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Pasaulio žinios. Klausimai patikrinti, ar nepablogėjo bendros žinios.

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb

# # Užtikriname, kad modelis yra tinkamame įrenginyje
sablonas = """ Žemiau yra pateikiamos instrukcijos ir atsakymai:

  ### Klausimas:
  {}

  ### Atsakymas:
  {}"""

# Įkeliame duomenis iš CSV failo
klausimai = pd.read_csv("pasaulio_zinios.csv")
istorija = []  # Saugojame pokalbio istoriją

# Kartojame per modelius (nuo 0 iki 5)
for f in range(0, 6, 1):
    # Įkeliame tokenizer'į
    tokenizer = AutoTokenizer.from_pretrained(f'./modelis_[{f}]')

    # Įkeliame modelį 4-bit tikslumu
    model = AutoModelForCausalLM.from_pretrained(
        f'./modelis_[{f}]',
        load_in_4bit=True,
        device_map="auto",
    )

    # Nustatome įrenginį (GPU arba CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Kartojame per visus klausimus iš duomenų rinkinio
    for i in range(len(klausimai)):
        # Paimame įvestį iš duomenų rinkinio
        user_input = klausimai["Input"][i]

        # Patikriname, ar įvesta "exit", norint išeiti
        if user_input.lower() == "exit":
            print("Pokalbis baigtas.")
            break

        # Paruošiame įvestį modeliui
        inputs = tokenizer(
            [
                sablonas.format(
                    user_input,  # Klausimas
                    ""  # Paliekame atsakymo vietą tuščią, kad modelis sugeneruotų
                )
            ],
            return_tensors="pt"
        ).to(device)

        # Generuojame modelio atsakymą
        output = model.generate(**inputs, max_new_tokens=30)

        # Dekoduojame atsakymą
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        assistant_response = response.split("### Atsakymas:")[-1].strip()

        # print(f"Klausimas {user_input} Asistentas: {assistant_response}")

        # Saugojame pokalbio istoriją
        istorija.append({
            "Klausimas": user_input,
            "Asistentas": assistant_response,
            "Tikrasis": klausimai["Output"][i]
        })

# Sukuriame duomenų rėmelį iš pokalbio istorijos
df = pd.DataFrame(istorija)

# Išsaugome istoriją į CSV failą
df.to_csv("pasaulio_zinios_istorija.csv")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Blue test

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu

# Sukuriame katalogą duomenų perkėlimui (offloading), jei jis dar neegzistuoja
offload_dir = "./offload"
os.makedirs(offload_dir, exist_ok=True)

# Įkeliame testavimo duomenų rinkinį
klausimai = pd.read_csv("14_apmokymo_klausimai.csv")  # Testo failas su 'Input' ir 'Output' stulpeliais
inputs = klausimai["Input"]
expected_outputs = klausimai["Output"]

# Sukuriame sąrašą rezultatams saugoti
istorija = []

# Apibrėžiame šabloną, skirtą formatuoti įvestį
sablonas = """ Žemiau yra pateikiama instrukcijos ir astakymai:

  ### Klausimas:
  {}

  ### Atsakymas:
  {}"""

# Kartojame per 3 modelius
for f in range(6):  # Ciklas per modelius 0, 1, 2
    # Įkeliame tokenizer'į ir modelį
    tokenizer = AutoTokenizer.from_pretrained(f'./modelis_[{f}]')
    model = AutoModelForCausalLM.from_pretrained(
        f'./modelis_[{f}]',
        device_map="auto",  # Automatiškai priskiriame sluoksnius CPU/GPU
        offload_folder=offload_dir,  # Nurodome katalogą duomenų perkėlimui
        load_in_4bit=True,  # Naudojame 4-bit tikslumą
    )

    # Užtikriname, kad modelis būtų tinkamame įrenginyje
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Saugojame prognozes ir BLEU vertes dabartiniam modeliui
    predictions = []
    bleu_scores = []

    # Kartojame per testavimo įvestis
    for i in range(len(klausimai)):
        # Paimame įvestį iš duomenų rinkinio
        user_input = klausimai["Input"][i]

        # Patikriname, ar vartotojas įvedė "exit" norėdamas išeiti
        if user_input.lower() == "exit":
            print("Pokalbis baigtas.")
            break

        # Suformuojame įvestį pagal šabloną
        formatted_input = sablonas.format(user_input, "")

        # Paruošiame įvestį modeliui
        inputs = tokenizer(
            [formatted_input],
            return_tensors="pt"
        ).to(device)

        # Generuojame atsakymą
        output = model.generate(inputs["input_ids"], max_new_tokens=30)

        # Dekoduojame atsakymą
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Išskiriame tik asistento atsakymą (po "### Atsakymas:")
        assistant_response = response.split("### Atsakymas:")[-1].strip()

        predictions.append(assistant_response)

        # Skaičiuojame BLEU vertę
        reference = [expected_outputs[i].split()]  # Tikras atsakymas, suskirstytas į žodžius
        candidate = assistant_response.split()  # Modelio atsakymas, suskirstytas į žodžius
        bleu_score = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu_score)

        # Klasifikuojame BLEU vertės kokybę
        if bleu_score <= 0.1:
            quality = "Labai bloga"
        elif bleu_score <= 0.3:
            quality = "Kai kurie sutapimai"
        elif bleu_score <= 0.5:
            quality = "Vidutiniai sutapimai"
        elif bleu_score <= 0.7:
            quality = "Geri sutapimai"
        else:
            quality = "Puikus atitikimas"

        # Saugojame pokalbio istoriją šiam modeliui
        istorija.append({
            "Modelis": f"Model_{f}",
            "Klausimas": user_input,
            "Prognozuotas": assistant_response,
            "Tikrasis": expected_outputs[i],
            "BLEU vertė": bleu_score,
            "Kokybė": quality
        })

    # Apskaičiuojame vidutinę BLEU vertę dabartiniam modeliui
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Vidutinė BLEU vertė modeliui Model_{f}: {avg_bleu:.4f}")

# Išsaugome rezultatus į CSV failą
results_df = pd.DataFrame(istorija)
results_df.to_csv("test_results_with_bleu_and_quality.csv", index=False)

print("Testavimo rezultatai išsaugoti į test_results_with_bleu_and_quality.csv")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score for Model_0: 0.0000


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

Average BLEU Score for Model_4: 0.0000


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score for Model_5: 0.0769
Test results saved to test_results_with_bleu_and_quality.csv


## Rough score naudojamas kad įvertinti, kaip apsimokė modelis

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb
from rouge_score import rouge_scorer

# Sukuriamas šablonas klausimams ir atsakymams
sablonas = """ Žemiau yra pateikiama instrukcijos ir astakymai:

  ### Klausimas:
  {}

  ### Atsakymas:
  {}"""

# Įkeliame duomenis iš CSV failo
klausimai = pd.read_csv("14_apmokymo_klausimai.csv")
rough_score = []  # Saugo pokalbio istoriją

for f in range(0, 6, 1):
    # Įkeliame tokenizer'į
    tokenizer = AutoTokenizer.from_pretrained(f'./modelis_[{f}]')

    # Įkeliame modelį su 4-bit kvantizacija
    model = AutoModelForCausalLM.from_pretrained(
        f'./modelis_[{f}]',
        load_in_4bit=True,
        device_map="auto",
    )

    # Nustatome, kurio įrenginio naudoti (GPU arba CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for i in range(len(klausimai)):
        # Paimame klausimą iš duomenų rinkinio
        user_input = klausimai["Input"][i]

        # Patikriname, ar vartotojas įvedė 'exit', norėdamas baigti
        if user_input.lower() == "exit":
            print("Pokalbis baigtas.")
            break

        # Paruošiame įvestį modeliui
        inputs = tokenizer(
            [
                sablonas.format(
                    user_input,  # Įvestas klausimas
                    ""  # Paliekame atsakymą tuščią, kad sugeneruotų modelis
                )
            ],
            return_tensors="pt"
        ).to(device)

        # Generuojame atsakymą
        output = model.generate(**inputs, max_new_tokens=30)

        # Dekoduojame modelio atsakymą
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        assistant_response = response.split("### Atsakymas:")[-1].strip()

        # Inicializuojame ROUGE vertinimo skaičiuoklę
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

        # Skaičiuojame ROUGE vertinimus
        scores = scorer.score(klausimai["Output"][i], assistant_response)

        # Saugojame pokalbio istoriją
        rough_score.append({
            "Modelis": f"modelis[{f}]",
            "Klausimas": user_input,
            "Asistentas": assistant_response,
            "Tikrasis": klausimai["Output"][i],
            "Rezultatai": [
                scores['rouge1'],
                # scores['rouge2'],
                # scores['rougeL']
            ]
        })

# Sukuriame duomenų rėmelį iš rezultatų ir išsaugome į CSV failą
df = pd.DataFrame(rough_score)
df.to_csv("rough_score.csv")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Pasibendraukime su Chatbot